# **Notebook for finding number of schools in 2km radius**

This notebook explores the relationship between the number of schools and various amenities within a 2km radius and the rental prices in different regions. Using geospatial data and statistical analysis, the notebook provides visualizations to interpret these relationships. It also investigates how sentiment and specific topics identified from the property descriptions affect rental prices.

Table of Contents

1. Data Loading and Preprocessing

    * Load property and school location data.
    * Create geospatial data frames for schools and properties.
    * Finding Schools within 2km Radius

2. Transform coordinate systems.
    * Spatial join to identify schools within 2km radius of properties.
    * Aggregate school counts by property.
    * Analysing Relationship between Schools and Rent Prices

3. Merge property and school count data.
    * Calculate average rent based on the number of schools within 2km.
    * Visualize rent price vs. school count with a line plot.
    * Statistical Analysis
    
4. Perform Pearson and Spearman correlation tests.
    * Conduct ANOVA to test the relationship between rent prices and school counts.
    * Visualizing Rental Properties and Amenities on a Map

5. Load and preprocess amenities data.
    * Visualize rental properties and different types of amenities on an interactive map.
    * Exploratory Analysis of Sentiment and Rent Prices

6. Box plots to show rent price distributions based on sentiment classification.
    * Analyze the relationship between sentiment and rent prices with density contour and trend line plots.
    * Rental Price Distribution by Topic Label

7. Visualize rental price distribution based on the dominant topic labels from the property descriptions.


In [45]:
import pandas as pd
import geopandas as gpd
import plotly.express as px  
import plotly.graph_objects as go  
from scipy.stats import linregress
import scipy.stats as stats  
import statsmodels.api as sm  
from statsmodels.formula.api import ols 

## **Feature: School count within 2km radius of properties**

In this section, we aim to enhance the dataset by creating a new feature that quantifies the number of schools within a 2km radius of each property. This will allow us to analyze the impact of nearby educational institutions on rental prices. The process involves the following steps:

1. Load Property and School Data: We start by loading both the property dataset and the school locations dataset. Each property has latitude and longitude coordinates, and the school dataset provides the location of schools in the area.

2. Geospatial Transformation: Both the property and school data are converted into GeoDataFrames, allowing us to perform spatial operations. The coordinate reference systems (CRS) are standardized to ensure that all data points are comparable.

3. Spatial Join to Find Schools within 2km: Using a spatial join, we calculate which schools fall within a 2km radius of each property. This is done by applying a 2km buffer around each property and checking if any schools fall within that buffer.

4. Aggregating the School Count: For each property, the number of schools within the 2km radius is counted and added as a new feature to the property dataset. Properties without any schools within the radius are assigned a count of zero.

5. Integration with Main Dataset: Finally, the newly created feature, School_Counts, is merged back into the main property dataset. This new feature allows us to explore correlations between the number of nearby schools and the rent prices in subsequent analysis.

In [36]:
properties = pd.read_csv("../data/curated/input_df.csv")

# Load the school location data from a CSV file. 'ISO-8859-1' encoding is used to handle special characters.
schools = pd.read_csv("../data/landing/dv346-schoollocations2023.csv",  encoding='ISO-8859-1')

# Convert the schools DataFrame into a GeoDataFrame, specifying the geometry from the 'X' and 'Y' columns 
schools_gdf = gpd.GeoDataFrame(schools, 
                               geometry=gpd.points_from_xy(schools['X'], schools['Y']),
                               crs="EPSG:4326")

geom_2km_gdf = gpd.GeoDataFrame(properties, geometry=gpd.GeoSeries.from_wkt(properties['geometr_2km']), crs="EPSG:3857")

schools_gdf = schools_gdf.to_crs(epsg=3857)
geom_2km_gdf = geom_2km_gdf.to_crs(epsg=3857)


In [37]:
# Spatial join between the schools GeoDataFrame and the properties' 2km geometry GeoDataFrame.
schools_within_2km = gpd.sjoin(schools_gdf, geom_2km_gdf, how="inner", predicate="within")

# Group the schools within 2km by the 'URL' of each property, and count the number of schools per property.
schools_count = schools_within_2km.groupby("URL").size()
schools_count = pd.DataFrame(schools_count)
schools_count = schools_count.reset_index()

schools_count


,URL,0
0,https://www.domain.com.au/-20-kosa-avenue-suns...,3
1,https://www.domain.com.au/-65-rosslyn-street-w...,9
2,https://www.domain.com.au/-leased-3-yarra-stre...,10
3,https://www.domain.com.au/006-903-dandenong-ro...,7
4,https://www.domain.com.au/04-390-burwood-highw...,3
...,...,...
11836,https://www.domain.com.au/unit-7-115-shaftesbu...,6
11837,https://www.domain.com.au/unit-9-2724-fourteen...,2
11838,https://www.domain.com.au/wheelers-hill-vic-31...,5
11839,https://www.domain.com.au/x03-342-346-centre-r...,7


In [38]:
schools_count.columns = ["URL", "School_Counts"]
merged_df = pd.merge(properties, 
                     schools_count,  
                     on='URL', 
                     how='left')

merged_df['School_Counts'] = merged_df['School_Counts'].fillna(0)
merged_df['School_Counts'].unique()
merged_df['Rent_Price'] = pd.to_numeric(merged_df['Rent_Price'], errors='coerce')
merged_df = merged_df.dropna(subset=['Rent_Price', 'School_Counts'])

average_rent_by_school_count = merged_df.groupby('School_Counts')['Rent_Price'].mean().reset_index()

# Line plot to show changes over increasing number of schools
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=average_rent_by_school_count['School_Counts'],
    y=average_rent_by_school_count['Rent_Price'],
    mode='lines+markers',
    name='Average Rent Price',
    line=dict(color='blue', width=3),
    marker=dict(size=8)
))

fig.update_layout(
    title='Average Rent Price vs Number of Schools within 2km',
    xaxis_title="Number of Schools within 2km",
    yaxis_title="Average Rent Price ($)",
    template="plotly_white",
    height=600,
    width=900
)

fig.show()


,URL,Rent_Price,Address,Bedrooms,Bathrooms,Parking,Property_Type,Latitude,Longitude,Date_Available,...,Intercom,Heating,Ensuite,Balcony / Deck,Fully fenced,Days_From_Now,negative_class,positive_class,geometr_2km,School_Counts
0,https://www.domain.com.au/7-isabella-way-tarne...,500.0,"7 Isabella Way, Tarneit VIC 3029",4,2,2,House,-37.837929,144.653245,Later,...,0,0,0,0,0,7,0.0,1.0,POLYGON ((16104725.59725854 -4556555.819541395...,2.0
1,https://www.domain.com.au/1906-570-lygon-stree...,460.0,"1906/570 Lygon Street, Carlton VIC 3053",1,1,0,Apartment / Unit / Flat,-37.793559,144.969056,Available Now,...,0,0,0,0,0,0,0.0,1.0,POLYGON ((16139881.53922936 -4550303.587526619...,10.0
2,https://www.domain.com.au/5-modra-street-point...,550.0,"5 Modra Street, Point Cook VIC 3030",4,2,2,House,-37.905707,144.773112,Available Now,...,0,0,0,0,0,0,0.0,1.0,POLYGON ((16118069.09726561 -4566113.910546225...,3.0
3,https://www.domain.com.au/5-63-droop-street-fo...,560.0,"5/63 Droop street, Footscray VIC 3011",2,1,1,Apartment / Unit / Flat,-37.796075,144.895730,Available Now,...,0,0,0,0,0,0,0.0,1.0,POLYGON ((16131718.92624745 -4550657.968199254...,6.0
4,https://www.domain.com.au/104-193-buckley-stre...,530.0,"104/193 Buckley Street, Essendon VIC 3040",2,1,1,Apartment / Unit / Flat,-37.756341,144.909078,Available Now,...,0,0,0,0,0,0,0.0,1.0,"POLYGON ((16133204.81881056 -4545061.96294156,...",11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12297,https://www.domain.com.au/21-hannah-street-ben...,155.0,"21 Hannah Street, Benalla VIC 3672",0,0,0,House,-36.543437,145.984015,Available Now,...,0,0,0,0,0,0,1.0,0.0,POLYGON ((16252866.24715346 -4375657.443938157...,3.0
12298,https://www.domain.com.au/34-yosemite-street-t...,530.0,"34 Yosemite Street, Truganina VIC 3029",4,2,2,House,-37.759074,144.696542,Available Now,...,0,0,0,0,0,0,0.0,1.0,POLYGON ((16109545.37498752 -4545446.804402602...,NaN
12299,https://www.domain.com.au/39-kenny-street-west...,550.0,"39 Kenny Street, Westmeadows VIC 3049",4,2,2,House,-37.676119,144.900169,Available Now,...,0,0,0,0,0,0,0.0,1.0,POLYGON ((16132213.04007123 -4533772.742955274...,5.0
12300,https://www.domain.com.au/311-1728-dandenong-r...,550.0,"311/1728 Dandenong Road, Clayton VIC 3168",2,1,0,Apartment / Unit / Flat,-37.913004,145.124261,Later,...,0,0,0,0,0,0,0.0,1.0,POLYGON ((16157158.83627013 -4567143.517090854...,4.0


In [46]:
pearson_corr, p_value_pearson = stats.pearsonr(merged_df['School_Counts'].astype(float), merged_df['Rent_Price'])

spearman_corr, p_value_spearman = stats.spearmanr(merged_df['School_Counts'].astype(float), merged_df['Rent_Price'])

# The formula 'Rent_Price ~ C(School_Counts)' means we are modeling rent prices as a function of school counts, treating 'School_Counts' as a categorical variable.
model = ols('Rent_Price ~ C(School_Counts)', data=merged_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print(anova_table)
print(f"Pearson Correlation: {pearson_corr}, p-value: {p_value_pearson}")
print(f"Spearman Correlation: {spearman_corr}, p-value: {p_value_spearman}")

                        sum_sq       df          F        PR(>F)
C(School_Counts)  7.626853e+06     17.0  21.750856  4.671476e-67
Residual          2.533728e+08  12284.0        NaN           NaN
Pearson Correlation: 0.12402895324474321, p-value: 2.2636622499709506e-43
Spearman Correlation: 0.12845832460461626, p-value: 2.0150789409760186e-46


#### **Conclusion for School counts and rent price**
1. Pearson Correlation: The Pearson correlation between the number of schools within 2km and the rent price is 0.124, indicating a weak positive linear relationship. The p-value is extremely small (2.26e-43), suggesting that this relationship is statistically significant.

2. Spearman Correlation: The Spearman rank correlation is 0.128, also indicating a weak positive monotonic relationship between school counts and rent prices. The very small p-value (2.02e-46) implies this relationship is highly significant as well.

3. ANOVA Test: The ANOVA results show a significant F-value (21.75) and a very small p-value (4.67e-67), suggesting that the differences in rent prices across various school count categories are statistically significant.



In [41]:
merged_df.to_csv("../data/curated/input.csv", index = False)

### **Further Visualisations of Features**

In [42]:
fig = px.box(
    properties,
    x='Dominant_Topic_Label',
    y='Rent_Price',
    color='Dominant_Topic_Label',
    title='Rental Price Distribution by Dominant Topic Label',
    labels={'Rent_Price': 'Rental Price', 'Dominant_Topic_Label': 'Dominant Topic Label'},
    template='plotly_white',
    color_discrete_sequence=px.colors.qualitative.Pastel
)

fig.update_layout(
    title={
        'text': 'Rental Price Distribution for Each Dominant Topic Label',
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='Dominant Topic Label',
    yaxis_title='Rental Price',
    showlegend=False
)

fig.show()


/Users/tinas/anaconda3/envs/group25env/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [43]:
properties['Class'] = properties.apply(lambda row: 'Negative' if row['negative_class'] == 1 else 'Positive', axis=1)

fig = px.box(
    properties,
    x='Class',
    y='Rent_Price',
    color='Class',
    title='Rental Price Distribution by Class',
    labels={'Rent_Price': 'Rental Price', 'Class': 'Class'},
    template='plotly_white',
    color_discrete_map={'Negative': 'red', 'Positive': 'green'}
)

fig.update_layout(
    title={
        'text': 'Rental Price Distribution for Negative and Positive Classes',
        'y': 0.9,
        'x': 0.5,
        'xanchor': 'center',
        'yanchor': 'top'
    },
    xaxis_title='Class',
    yaxis_title='Rental Price',
    showlegend=False
)

fig.show()

/Users/tinas/anaconda3/envs/group25env/lib/python3.11/site-packages/plotly/express/_core.py:1979: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [47]:
fig = px.density_contour(
    properties, 
    x="Sentiment", 
    y="Rent_Price", 
    title="Density Contour with Trend Line: Sentiment vs Rent Price", 
    labels={'Sentiment': 'Sentiment', 'Rent_Price': 'Rent Price (AUD)'},
    template='plotly_white'
)

fig.update_traces(contours_coloring="fill", colorscale="thermal")

# Calculate the linear regression line using scipy's linregress
slope, intercept, _, _, _ = linregress(properties['Sentiment'], properties['Rent_Price'])
x_vals = np.array(properties['Sentiment'])
y_vals = intercept + slope * x_vals

fig.add_trace(
    go.Scatter(
        x=x_vals,
        y=y_vals,
        mode='lines',
        name='Trend Line',
        line=dict(color='red', width=2)
    )
)

fig.show()